# Split dataset and upload to huggingface
* This notebook will spilt the dataset into: training, validate and test set.
* Then upload the dataset to huggingface 

In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from huggingface_hub import notebook_login

In [2]:
name = "high_vs_random"
file_name = f"csv/clean_{name}.csv"
df = pd.read_csv(file_name)
df

,Unnamed: 0,text_clean,label
0,0,tools mode layout issuesdescribe the bug some ...,0
1,1,version control enhancement for osimagephase 2...,0
2,2,actor future is immediately completed if the a...,1
3,3,pub should more efficiently serialize barback ...,0
4,4,hu editor hu status different in grid view sin...,1
...,...,...,...
116797,116797,qgis 310 ltr 316 not connecting to odbc source...,1
116798,116798,active theme is not foundhello every 15 second...,1
116799,116799,crash at usedproduct report creation scenario ...,1
116800,116800,optimization api 在线文档在生产环境隐藏加权限 permissionclas...,0


In [6]:
df.label.value_counts().to_frame()[:50]
value_counts_df = df['label'].value_counts().to_frame()
value_counts_df

,count
label,
0,58401
1,58401


In [7]:
# Split dataframe into three parts: training, validation and testing.
# Currently 80/10/10 split.
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=42):
    np.random.seed(seed)
    # Shuffle index of dataframe
    perm = np.random.permutation(df.index)
    
    df_length = len(df.index)
    
    # Number of row in training set
    train_end = int(train_percent * df_length)
    # Number of rows in validate set
    validate_end = int(validate_percent * df_length) + train_end
    
    # From start to train end
    train = df.iloc[perm[:train_end]]
    # From train_end to validate_end
    validate = df.iloc[perm[train_end:validate_end]]
    # From validate to the last row in dataframe.
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [8]:
# Split dataframe into three parts: training, validation and testing.
train , validate , test = train_validate_test_split(df)

In [9]:
# Convert from Pandas DataFrame to Hugging Face datasets
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
test_ds = Dataset.from_pandas(test)

ds = DatasetDict()

ds["test"] = test_ds
ds["train"] = tds
ds["validate"] = vds

ds

DatasetDict({
    test: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'label', '__index_level_0__'],
        num_rows: 11681
    })
    train: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'label', '__index_level_0__'],
        num_rows: 93441
    })
    validate: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'label', '__index_level_0__'],
        num_rows: 11680
    })
})

In [ ]:
# Uncomment this if not logged in.
#notebook_login()

In [10]:
# Push to Hugging Face Hub
ds.push_to_hub(name+"min_100_issues_per_repo")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/94 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]